In [1]:
from skimage.feature import hog
import numpy as np
import cv2

In [2]:
from dask import delayed
import dask

In [3]:
def get_hog_features(
    img, orient, pix_per_cell, cell_per_block, feature_vec = True
):

    features = hog(
        img,
        orientations = orient,
        pixels_per_cell = (pix_per_cell, pix_per_cell),
        cells_per_block = (cell_per_block, cell_per_block),
        transform_sqrt = True,
        visualise = False,
        feature_vector = feature_vec,
    )
    return features


def bin_spatial(img, size = (16, 16)):
    return cv2.resize(img, size).ravel()


def img_features(
    feature_image, hist_bins, orient, pix_per_cell, cell_per_block, spatial_size
):
    features = []
    features.append(bin_spatial(feature_image, size = spatial_size))
    features.append(
        get_hog_features(feature_image, orient, pix_per_cell, cell_per_block)
    )
    return features


def extract_features(img):
    img = cv2.resize(img, (350, 125))
    file_features = img_features(
        img,
        hist_bins = 32,
        orient = 8,
        pix_per_cell = 8,
        cell_per_block = 2,
        spatial_size = (32, 32),
    )
    return np.concatenate(file_features)

In [4]:
import pickle

with open('detect-dataset.pkl', 'rb') as fopen:
    dataset = pickle.load(fopen)

In [5]:
dataset.keys()

dict_keys(['positive', 'negative'])

In [6]:
from tqdm import tqdm

positives = []
for i in tqdm(range(len(dataset['positive'][:10000]))):
    r = extract_features(dataset['positive'][i])
    positives.append(r)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/husein/.local/lib/python3.6/site-packages/skimage/feature/_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)
100%|██████████| 10000/10000 [04:01<00:00, 41.38it/s]


In [7]:
from tqdm import tqdm

negatives = []
for i in tqdm(range(len(dataset['negative'][:10000]))):
    r = extract_features(dataset['negative'][i])
    negatives.append(r)

100%|██████████| 10000/10000 [04:01<00:00, 41.34it/s]


In [8]:
import pickle

with open('features.pkl', 'wb') as fopen:
    pickle.dump({'positive': positives, 'negative': negatives}, fopen)